<a href="https://colab.research.google.com/github/lnbentes/ImersaoAluraIAGemini/blob/main/Imers%C3%A3oAlura3_Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Esse projeto visa criar um bot no qual utilizamos agentes para poder processar uma list de compra fornecida pelo usuária, identificar os supermercado ao redor do endereço fornecido e retorna a lista com os melhores preços da sua região.

In [2]:
# podemos usar o !pip para realizar instalacoes em python
!pip install -q google-genai

In [3]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY_ALURA')

In [ ]:
!pip install -q google-adk #-q não mostra todo o texto da instalação

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings
import ast

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final (Ele e o arquestrador)
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [35]:
###############################################
# --- Agente 1: Buscador de Supermecardos --- #
###############################################
def agente_buscador_supermercado(endereco, data_de_hoje):
    buscador_supermercado = Agent(
        name="agente_buscador",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para localizar os suplermercados perto da localização do usuario e retornara o nome dos três mais pertos.

        Contexto:
        retorne uma lista somente com os nomes dos supermercados;
        Na lista senha um chave de nome e endereço;
        Se não tiver endereço coloque -;
        Seja direto e sucinto;
        Apresnete comente a lista e mais nada;

        Exemplo de retorno:
        Nome: Supermercado Carone Endereço na Av. Hugo Viola, 615, Jardim da Penha.
        Nome: Supermercados BH Endereço: Pça Regina Frigeri Furno, 210, Jardim da Penha.
        Nome: Extraplus de Jardim da Penha Endereço: -
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador_supermercado = f"Localização/Endereço: {endereco}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador_supermercado, entrada_do_agente_buscador_supermercado)

    return lancamentos

In [60]:
######################################
# --- Agente 2: Formatar a lista --- #
######################################
def agente_formatador(lista):
    formatador = Agent(
        name="agente_formatador",
        model="gemini-2.0-flash",
        instruction="""
            Você é um experiente proficional que pega uma lista de texto e converte ela no estilo list em python.

            Contexto:
            retorne uma lista sestile list em python;
            Seja direto e sucinto na analise e resposta;
            Não precisa mostrar a sua análise, somente a resposta do retorno;
            Não precisa adicionar texto como Python antes da lista;
            Retorne somente a lista, sem texto adicionais;

            Exemplo 1 do que não deve retornar:
            ```python
            ['Feijão', 'Shampoo', 'Ferro de Passar', 'Sabão em pó', 'Amaciante', 'Biscoito recheado', 'Picanha 1kg', 'Arroz tipo 1']
            ```
            Exemplo 2 de que não deve retornar:
            python ['Barbosa Supermercados Loja São Judas', 'Supermercados São Judas Tadeu', 'Supermercado Porto Seguro Sao Judas']

            Exemplo 1 de retorno correto:
            ['Feijão', 'Shampoo', 'Ferro de Passar', 'Sabão em pó', 'Amaciante', 'Biscoito recheado', 'Picanha', 'Ervilha']
            Exemplo 2 de retorno correto:
            ['Barbosa Supermercados Loja São Judas', 'Supermercados São Judas Tadeu', 'Supermercado Porto Seguro Sao Judas']
            """,
        description="Agente de converter lista em Python"
    )

    entrada_do_agente_formatador = f"Lista: {lista.lower()}"

    rascunho = call_agent(formatador, entrada_do_agente_formatador)

    return rascunho.lower()

In [63]:
#########################################
# --- Agente 3: Buscador dos preços --- #
#########################################
def agente_buscador_preco(itens, supermercados, endereco):
    texto_final = ""

    buscador_preco = Agent(
        name="agente_buscador_preco",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para buscar o preço de um item no supermercado indicado pelo usuário.

        Contexto:
        Retorne somente o nome do produto com o seu preço;
        Se o produto estiver com o nome muito genérico, pesquise o a marca mais popular para esse item;
        Seja direto e sucinto na analise e resposta;
        Não precisa mostrar a sua análise, somente a resposta do retorno;
        Em dinheiro use a formatação Brasileira para moedas;

        Exemplo e passo a passo:
        1- Verificar se o item está com o nome genérico; Exemplo: chocolate - nome generico
        2- Pesquisar no google a marca mais popular para esse item; Exemplo: chocolate popular em vitoria - Chocolate GAROTO ao Leite 80g
        3- Pesquisar se o valor do produto no supermercado; Exemplo: Preço do Chocolate GAROTO ao Leite 80g no supermercado Carone - R$ 6,99
        4- Retornar o nome do produto com o seu preço; Exemplo: Chocolate GAROTO ao Leite 80g - R$ 6,99
        5- Se não achar o valor do item para o supermercado em questão, pesquise para a Loja Lojas Americanas; Exemplo: Preço do Chocolate GAROTO ao Leite 80g na Lojas Americanas - R$ 11,85

        Exemplo de retorno pelo supermercado:
        Chocolate GAROTO ao Leite 80g - R$ 6,99 (no Supermercado Carone)

        Exemplo de retorno pela Lojas Americanas:
        Preço do Chocolate GAROTO ao Leite 80g - R$ 11,85 (na Lojas Americanas)

        Exemplo de retorno final
        Chocolate GAROTO ao Leite 80g - R$ 6,99 (no Supermercado Carone)

        """,
        description="Agente que busca preços de produtos no Google",
        tools=[google_search]
    )

    try:
      lista_itens = ast.literal_eval(itens)

      lista_supermercados = ast.literal_eval(supermercados)

      if(type(lista_itens) == list and type(lista_supermercados) == list):
        for supermercado in lista_supermercados:

          texto_final = texto_final + f"\n +++ Supermercado: {supermercado} +++\n"

          for item in lista_itens:
            entrada_do_agente_buscador_preco = f"Item:{item}\nSupermercado: {supermercado}\nLocalização/Endereço do supermercado: {endereco.lower()}"

            plano_do_post = call_agent(buscador_preco, entrada_do_agente_buscador_preco)

            plano_do_post_minuscilo = plano_do_post.lower()

            texto_final = texto_final + f"Item: {item} \n- Item/Preço presquisado: {plano_do_post_minuscilo}\n\n"
      else:
        texto_final = "Erro na busca dos itens"
      pass
    except:
      texto_final = "Erro na busca dos itens"
    finally:
        return texto_final
        pass

In [38]:
#######################################################
# --- Agente 4: Escolher a lista com melhor preço --- #
#######################################################
def agente_analista(lista):
    analista = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Analista especializado em sempre slecionar a melhor lista de compras de suplermercado,
            voce recebe uma lista de listas de compras e retorna a melhor lista.

            Contexto:
            Você recebera ums lista com varias listas de compras de supermercados diferentes;
            Cada subi lista de supermercado está os itens com o seus valores em moeda Real;
            Ira analizar o valor dos itens de um supermercado e comprar com os itens de outro supermercado e selecionar o supermercado que tem o maior numero de itens com o menor valor;
            Contudo se muitos dos itens dos supermercados forem de outro local, crie uma nova lista so com os itens externos;
            Seja direto e sucinto na análise e resposta;
            Não precisa mostra a sua análise, somente a resposta do retorno;
            Se a lista de entrada tiver o texto "Erro na busca dos itens" crie uma lista de compras vazia

            Exemplo de resposta:
            **Nova Lista (Itens Externos):**
            *   Feijão Carioca Kicaldo 1kg - R$ 6,99 (RedeMix)
            *   Arroz Tio João Tipo 1 1kg - R$ 7,99 (Hortifruti)
            *   Detergente Ypê Neutro 500ml - R$ 3,28 (Lojas Americanas)

             Exemplo de resposta se tiver o texto "Erro na busca dos itens":
            **Lista Nula ("Erro na busca dos itens"):**
            *   item 1 - R$ xx,xx (xxx)
            *   item 2 - R$ xx,xx (xxx)
            *   item 3 - R$ xx,xx (xxx)

            """,
        description="Agente analista ima lista de compras e escolhe qual tem o melhor preço"
    )

    entrada_do_agente_analista = f"Lista: {lista}"

    rascunho = call_agent(analista, entrada_do_agente_analista)
    return rascunho

In [57]:
##########################################
# --- Agente 5: Revisor de Qualidade --- #
##########################################
def agente_revisor(rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em criar lista de compras e deixalas bem formatadas.
            Revise o rascunho de uma lista de compras verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.

            Contexto:
            Caso haja problemas, formate ele deixando mais legível;
            Em dinheiro use a formatação Brasileira para moedas;

            Exemplo de formato:
            Nova Lista (Itens Externos):
            *   Feijão Carioca Kicaldo 1kg - R$ 6,99 (RedeMix)
            *   Arroz Tio João Tipo 1 1kg - R$ 7,99 (Hortifruti)
            *   Detergente Ypê Neutro 500ml - R$ 3,28 (Lojas Americanas)
            """,
        description="Agente revisor de lista de compras."
    )

    entrada_do_agente_revisor = f"Rascunho: {rascunho_gerado}"

    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)

    return texto_revisado



---
# Bloco de teste


In [ ]:
###### Bloco dos teste ######
# Teste agente 1

print("Inicio da execução do Agente 1 \n")
print(agente_buscador_supermercado("Vitória, ES - jardim camburui",  date.today().strftime("%d/%m/%Y")))
print("Fim da execução do Agente 1 \n")

# Teste agente 2

print("Inicio da execução do Agente 2 \n")
lista = """
Feijão Shampoo Ferro de Passar Sabão em pó Amaciante Biscoito recheado Picanha 1kg Arroz tipo 1
"""
print(agente_formatador(lista))
print("Fim da execução do Agente 2 \n")

# Teste agente 3

print("Inicio da execução do Agente 3 \n")

supermercados = """
['Supermercado Carone', 'Epa Boa Praça Supermercados']
"""

itens = """
['Feijão', '1kg Arroz tipo 1', 'detergente de loucça neutro 500ml']
"""

endereco = "Vitória, ES - jardim camburui"

print(agente_buscador_preco(itens, supermercados, endereco))
print("Fim da execução do Agente 3 \n")

# Teste agente 4

print("Inicio da execução do Agente 4 \n")

supermercados = """
##### Supermercado: Supermercado Carone #####
# Supermercado: Supermercado Carone - Item: Feijão
- Item/Preço presquisado: Não encontrei o preço do feijão no Supermercado Carone.
Feijão Carioca Kicaldo 1kg - R$ 6,99 (na RedeMix)
# Supermercado: Supermercado Carone - Item: 1kg Arroz tipo 1
- Item/Preço presquisado: Arroz Tio João Tipo 1 1kg - R$ 7,99 (Hortifruti)
# Supermercado: Supermercado Carone - Item: detergente de loucça neutro 500ml
- Item/Preço presquisado: Considerando a popularidade da marca, buscarei o preço do detergente Ypê Neutro 500ml.
Detergente Ypê Neutro 500ml - Indisponível no Supermercado Carone.
Detergente Ypê Neutro 500ml - R$ 3,28 (na Lojas Americanas)
##### Supermercado: Epa Boa Praça Supermercados #####
# Supermercado: Epa Boa Praça Supermercados - Item: Feijão
- Item/Preço presquisado: Feijão Carioca Camil 1kg - R$ 6,99 (no Boa Supermercados)
# Supermercado: Epa Boa Praça Supermercados - Item: 1kg Arroz tipo 1
- Item/Preço presquisado: Arroz Sepé Tipo 1 1kg - R$ 8,99 (Preço aproximado na Lojas Americanas)
# Supermercado: Epa Boa Praça Supermercados - Item: detergente de loucça neutro 500ml
- Item/Preço presquisado: O detergente de louça neutro mais popular é o Ypê.
Detergente Ypê Neutro 500ml - R$ 3,49 (Natural da Terra)
Não foi possível encontrar o preço no Epa Boa Praça Supermercados.
"""

print(agente_analista(supermercados))

print("Fim da execução do Agente 4 \n")

# Teste agente 5

print("Inicio da execução do Agente 5 \n")

rascunho = """
**Nova Lista (Itens Externos):**
*   Feijão Carioca Kicaldo 1kg - R$ 6,99 (RedeMix)
*   Arroz Tio João Tipo 1 1kg - R$ 7,99 (Hortifruti)
*   Detergente Ypê Neutro 500ml - R$ 3,28 (Lojas Americanas)
"""

print(agente_revisor(rascunho))

print("Fim da execução do Agente 5 \n")



---
# Bloco Main


In [66]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🗒️ Iniciando o Sistema de Buscar o Melhor Preço das Compras com 5 Agentes 💸")

# --- Obter o Tópico do Usuário ---
endereco = input("Para iniciar vamos precidar de duas coisas, a primeira e o seu endereço: ")
lista = input("Agora precisamos da sua lista de compras: ")

# Inserir lógica do sistema de agentes ################################################
if not endereco and not lista:
    print("Você esqueceu o endereço ou a lista de compras! Tente novamente")
else:
    print(f"Maravilha! Vamos então vasculhando por melhores preços pela região {endereco}")

    buscador_supermercado = agente_buscador_supermercado({endereco},  date.today().strftime("%d/%m/%Y"))
    print("\n--- 📝 Resultado do Agente 1 (Buscador de Supermercado) ---\n")
    display(to_markdown(buscador_supermercado))
    print("--------------------------------------------------------------")

    formatador_lista_compras = agente_formatador(lista)
    formatador_lista_supermercados = agente_formatador(buscador_supermercado)
    print("\n--- 📝 Resultado do Agente 2 (Formatador) ---\n")
    display(to_markdown(f"Lista de compras: {formatador_lista_compras} - Lista de supermercados: {formatador_lista_supermercados}"))
    print("--------------------------------------------------------------")

    buscador_preco = agente_buscador_preco(formatador_lista_compras, formatador_lista_supermercados, endereco)
    print("\n--- 📝 Resultado do Agente 3 (Buscador de preco) ---\n")
    print(buscador_preco)
    print("--------------------------------------------------------------")

    analise = agente_analista(buscador_preco)
    print("\n--- 📝 Resultado do Agente 4 (Analista) ---\n")
    print(analise)
    print("--------------------------------------------------------------")

    revisor = agente_revisor(analise)
    print("\n--- 📝 Resultado do Agente 5 (Revisor) ---\n")
    print(revisor)
    print("--------------------------------------------------------------")

🗒️ Iniciando o Sistema de Buscar o Melhor Preço das Compras com 5 Agentes 💸
Para iniciar vamos precidar de duas coisas, a primeira e o seu endereço: São Judas, São Paulo - Brasil
Agora precisamos da sua lista de compras: Arroz Feijão Biscoito Coca cola zero 2L
Maravilha! Vamos então vasculhando por melhores preços pela região São Judas, São Paulo - Brasil

--- 📝 Resultado do Agente 1 (Buscador de Supermercado) ---



> Nome: Barbosa Supermercados Endereço: AV COMENDADOR ANTUNES DOS SANTOS, 750, JARDIM SAO JUDAS TADEU, São Paulo-SP.
> Nome: Supermercados São Judas Tadeu Endereço: -
> Nome: Supermercado Estrela de Regente Feijo Endereço: Avenida Regente Feijó, 445, SUPERMERCADO ESTRELA - Centro - Regente Feijó - SP.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Formatador) ---



> Lista de compras: ['arroz', 'feijão', 'biscoito', 'coca cola zero 2l']
> 
>  - Lista de supermercados: ['barbosa supermercados', 'supermercados são judas tadeu', 'supermercado estrela de regente feijó']


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Buscador de preco) ---


 +++ Supermercado: barbosa supermercados +++
Item: arroz 
- Item/Preço presquisado: arroz raroz t1 5kg - r$ 16,99 (no barbosa supermercados).


Item: feijão 
- Item/Preço presquisado: feijão carioca kicaldo 1kg - r$ 7,69 (no quartetto supermercados).
feijão carioca kicaldo 1kg - r$ 7,99 (no supermercado san sato).
feijão carioca kicaldo 1kg - r$ 6,99 (no supermercado pague menos).


Item: biscoito 
- Item/Preço presquisado: biscoito negresco coberto chocolate 120g - r$ 7,99 (no barbosa supermercados).


Item: coca cola zero 2l 
- Item/Preço presquisado: coca-cola zero 2l - r$ 11,99 (no supermercado barbosa).



 +++ Supermercado: supermercados são judas tadeu +++
Item: arroz 
- Item/Preço presquisado: arroz empório são joão - r$ 25,00 (no supermercados são judas tadeu)


Item: feijão 
- Item/Preço presquisado: feijão carioca kicaldo 1kg - r$ 6,99 (no supermercado são ju